# TF-IDF를 활용한 모델 구현

## 데이터 불러오기

In [3]:
import pandas as pd

In [4]:
DATA_IN_PATH = './data_in/' 
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_DATA )

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

## TF-IDF 벡터화

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000) 
# min_df - 설정한 값보다 특정 토큰의 df 값이 더 적게 나오면 벡터화 과정에서 제거
# analyzer - 분석하가 위한 기준 단위 'word' or 'char'
# sublinear_tf - 문서의 단어 빈도 수에 대한 스무딩 여부
# ngram_range - 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인지를 보는 인자
# max_features - 각 벡터의 최대 길이, 특징의 길이를 설정

X = vectorizer.fit_transform(reviews)

## 학습과 검증 데이터셋 분리

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np

RANDOM_SEED = 42
TEST_SPLIT = 0.2

y = np.array(sentiments)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

## 모델 선언 및 학습

In [9]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced') # class_weight='balanced' - 각 라벨에 대해 균형 있게 학습
lgs.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

## 검증 데이터로 성능 평가

In [10]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

Accuracy: 0.859800


## 데이터 제출하기



### 평가 데이터 불러오기

In [14]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

### TF-IDF 값으로 벡터화

In [15]:
testDataVecs = vectorizer.transform(test_data['review'])

### 형태 확인

In [16]:
test_predicted = lgs.predict(testDataVecs)

print(test_predicted)

[1 0 1 ... 0 1 1]


### 제출 파일 만들기

In [19]:
DATA_OUT_PATH = './data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)